# Predicting Titanic Passenger Survival

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Exploratory data analysis (inspect dataset, evaluate distribution of variables to find possible correlations)
### Data cleaning to handle/remove missing values
### Build model and test

In [ ]:
training = pd.read_csv('/kaggle/input/titanic/train.csv')
training.info(verbose=True)


In [ ]:
training.describe()

In [ ]:
training.describe().columns

In [ ]:
df_num = training[['Age', 'SibSp', 'Parch', 'Fare']]
df_cat = training[['Survived', 'Pclass', 'Sex', 'Ticket', 'Cabin', 'Embarked']]

In [ ]:
for i in df_num.columns:
    plt.hist(df_num[i])
    plt.title([i])
    plt.show()

In [ ]:
print(df_num.corr())
sns.heatmap(df_num.corr())

In [ ]:
pd.pivot_table(training, index = 'Survived', values = ['Age', 'SibSp', 'Parch', 'Fare'])

In [ ]:
for i in df_cat.columns:
    sns.barplot(x=df_cat[i].value_counts().index, y=df_cat[i].value_counts())
    plt.title(i)
    plt.show()

In [ ]:
print(pd.pivot_table(training, index = 'Survived', columns = 'Pclass', values = 'Ticket', aggfunc = 'count'))
print()
print(pd.pivot_table(training, index = 'Survived', columns = 'Sex', values = 'Ticket', aggfunc = 'count'))
print()
print(pd.pivot_table(training, index = 'Survived', columns = 'Embarked', values = 'Ticket', aggfunc = 'count'))

For the sake of time, and because it's a bit beyond my own knowledge, I skipped the feature engineering for cabin letter/ticket letter/name, so I'm not including those in my model.

1) Drop null values from embarked
2) include only relevant variables ('Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked')
3) Categorical transforms on training and test dataset (see original video for admonitions against doing this outside competition environment)
4) Impute data with mean or median for fare and age
5) Normalize distribution for fare using log
6) Scale data 0-1 with standard scaler